In [1]:
from typing import Self
from dataclasses import dataclass
from tqdm import tqdm

In [2]:
filename = "sample.txt"
# filename = "sample2.txt"
# filename = "sample3.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

raw_grid, raw_moves = data.strip().split("\n\n")
grid_lines = raw_grid.split("\n")
moves = raw_moves.replace("\n", "")

https://adventofcode.com/2024/day/15
- Definitely overcomplicated this one. Probably would have been simpler to hard-code BigBox linking and movement (e.g. store as 1 Entity with 2 positions?)
- Baba is You, Sokoban is Pain

In [3]:
start_pos = None
walls = set()
start_boxes = set()
for y, line in enumerate(grid_lines):
    for x, c in enumerate(line):
        pos = complex(x, y)
        match c:
            case '#':
                walls.add(pos)
            case '@':
                start_pos = pos
            case 'O':
                start_boxes.add(pos)
            case '.':
                pass
            case _:
                print(f"Unknown character {c=} at {pos=}")


In [4]:
@dataclass
class Entity:
    pos: complex
    symbol: str
    
    def move(self, grid: dict[complex, Self], next_pos: complex):
        # Move without checking
        grid[next_pos] = self
        del grid[self.pos]
        self.pos = next_pos

    def is_pushable(self, grid: dict[complex, Self], step: complex) -> bool:
        # For part 2: Look before we move the boxes
        next_pos = self.pos + step
        if next_pos == self.pos:
            raise NotImplementedError(f"{self} planning move of {step} from {self.pos} and stayed in place")
        
        # Check if next pos is occupied. If it is, try and push it
        if next_pos in grid:
            push_successful = grid[next_pos].is_pushable(grid, step)
            if not push_successful:
                return False
        # There's free space, can move into that spot
        return True
    
    def push(self, grid: dict[complex, Self], step: complex) -> bool:
        next_pos = self.pos + step
        if next_pos == self.pos:
            raise NotImplementedError(f"{self} moved {step} from {self.pos} and stayed in place")
        
        # Check if next pos is occupied. If it is, try and push it
        if next_pos in grid:
            push_successful = grid[next_pos].push(grid, step)
            if not push_successful:
                return False
            
        # There's free space, move into that spot
        self.move(grid, next_pos)
        return True
    
    def gps(self) -> int:
        return 0

@dataclass
class Wall(Entity):
    symbol: str = '#'
    
    def is_pushable(self, grid: dict[complex, Entity], step: complex) -> bool:
        return False

    def push(self, grid: dict[complex, Entity], step: complex) -> bool:
        # Can't push a wall
        return False
    
@dataclass
class Box(Entity):
    symbol: str = 'O'
    
    def gps(self) -> int:
        return int(self.pos.real + 100 * self.pos.imag)
    
@dataclass
class BigBox(Entity):
    symbol: str = 'B'
    linked: list[Self] = None

    def __post_init__(self):
        if self.linked is None:
            self.linked = []

    def _linked_positions(self):
        return [e.pos for e in self.linked]
    
    def gps(self) -> int:
        if self.symbol == '[':
            return int(self.pos.real + 100 * self.pos.imag)
        return 0

    def is_pushable(self, grid: dict[complex, Entity], step: complex, *, check_linked=True) -> bool:
        # For part 2: Look before we move the boxes
        next_pos = self.pos + step
        if next_pos == self.pos:
            raise NotImplementedError(f"{self} planning move of {step} from {self.pos} and stayed in place")
        
        # Check if next pos is occupied. If it is, try and push it
        # If next pos is part of this Big Box, skip this step - if that half can move, then so can this
        if (next_pos in grid) and (next_pos not in self._linked_positions()):
            push_successful = grid[next_pos].is_pushable(grid, step)
            if not push_successful:
                return False
            
        # Make sure all linked parts to this box can move too
        if check_linked:
            for box in self.linked:
                push_successful = box.is_pushable(grid, step, check_linked=False)
                if not push_successful:
                    return False
        # There's free space, can move into that spot
        return True
    

    def push(self, grid: dict[complex, Self], step: complex, *, push_linked=True) -> bool:
        next_pos = self.pos + step
        if next_pos == self.pos:
            raise NotImplementedError(f"{self} moved {step} from {self.pos} and stayed in place")
        
        # Check if next pos is occupied. If it is, push it
        # If next pos is part of this Big Box, skip - don't push it twice
        if (next_pos in grid) and (next_pos not in self._linked_positions()):
            push_successful = grid[next_pos].push(grid, step)
            if not push_successful:
                return False

        # Push linked boxes
        if push_linked:
            for box in self.linked:
                push_successful = box.push(grid, step, push_linked=False)
                if not push_successful:
                    return False
                        
        # There's free space, move into that spot
        # ~~Note: self may overlap with linked boxes during this step~~ # TODO check?
        self.move(grid, next_pos)

        return True

@dataclass
class Robot(Entity):
    symbol: str = '@'

@dataclass
class Warehouse:
    grid: dict[complex, Entity]
    robot: Robot

    def move_robot(self, step: complex, *, check_first: bool = False) -> bool:
        if check_first:
            pushable = self.robot.is_pushable(self.grid, step)
            if not pushable:
                return False
        return self.robot.push(self.grid, step)
    
    def gps(self) -> int:
        return sum(e.gps() for e in self.grid.values())

def create_entities(walls, boxes, robot, big_boxes=[]) -> Warehouse:
    grid = dict()
    for pos in walls:
        grid[pos] = Wall(pos)
    for pos in boxes:
        grid[pos] = Box(pos)
    for pos1, pos2 in big_boxes:
        box_left = BigBox(pos1, '[')
        box_right = BigBox(pos2, ']')
        box_left.linked.append(box_right)
        box_right.linked.append(box_left)
        grid[pos1] = box_left
        grid[pos2] = box_right
    grid[robot] = Robot(robot)
    return Warehouse(grid, grid[robot])

def visualise(grid, x_hi, y_hi):
    for y in range(y_hi):
        for x in range(x_hi):
            entity = grid.get(complex(x, y))
            symbol = entity.symbol if entity else '.'
            print(symbol, end="")
        print()

In [5]:
def parse_step(c: str) -> complex:
    match c:
        case '>':
            return 1
        case 'v':
            return 1j
        case '<':
            return -1
        case '^':
            return -1j


In [6]:
## Part 1
warehouse = create_entities(walls, start_boxes, start_pos)
x_hi, y_hi = len(grid_lines[0]), len(grid_lines)
for i, c in enumerate(tqdm(moves), 1):
    step = parse_step(c)
    warehouse.move_robot(step)

visualise(warehouse.grid, x_hi, y_hi)
warehouse.gps()

100%|██████████| 700/700 [00:00<00:00, 732355.40it/s]

##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########


10092

In [7]:
## Part 2
# Big warehouse! Everything except the robot is 2x as big
# Big boxes can push 2 others at the same time
# Note: All the boxes stay in place if any is blocked!
p2_walls = {complex(pos.real * 2 + off, pos.imag) for pos in walls for off in (0, 1)}
p2_boxes = {tuple(complex(pos.real * 2 + off, pos.imag) for off in (0, 1)) for pos in start_boxes}
p2_start_pos = complex(start_pos.real * 2, start_pos.imag)
p2_warehouse = create_entities(p2_walls, [], p2_start_pos, p2_boxes)
for i, c in enumerate(tqdm(moves), 1):
    step = parse_step(c)
    p2_warehouse.move_robot(step, check_first=True)

visualise(p2_warehouse.grid, x_hi * 2, y_hi)
p2_warehouse.gps()

100%|██████████| 700/700 [00:00<00:00, 420692.48it/s]

####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################


9021